#Querying Documentum from a Jupyter Notebook
_(c) 2019 MSRoth_

This notebook demostrates how to interface with the Documentum Content Server from a Jupyter Notebook using a Java kernel and the DFC.  Enter any valid query below and see the results rendered inline.

This notebook utilizes the iJava Kernel (https://github.com/SpencerPark/IJava), the Documentum Foundation Classes (DFC), and two helper classes:  DCTMBasics (https://github.com/msroth/dmBasics), and dmRecordSet (https://github.com/msroth/dmRecordSet).



_Use some Jupyter magic to set classpath.  Specifically, add the location of the dfc.properties file, the dctm.jar file, and the two helper classes to the classpath.  Adjust these paths as required for your environment._

In [ ]:
%classpath "C:/Documentum/config"
%classpath "C:/Program Files/Documentum/dctm.jar"
%classpath "C:/Documents/Jupyter Notebooks/DCTM/*.jar"

import com.dm_misc.dctm.DCTMBasics;
import com.dm_misc.collections.dmRecordSet;
import com.documentum.fc.client.*;
import com.documentum.fc.common.*;

In [ ]:
// Enter username, password, and docbase values below.
String user = "dmadmin";
String password = "dmadmin";
String docbase = "repo1";

// It takes a few seconds to logon... wait for session Id before continuing
System.out.print("Logging on...");
IDfSession session = DCTMBasics.logon(docbase, user, password);
if (session != null) { 
    System.out.println("Session Id: " + session.getSessionId());
} else {
    System.out.println("Could not logon");
}

In [ ]:
// Enter a valid query string
String dql = "select * from dm_sysobject where folder('/Temp')";

In [ ]:
// Run the query
IDfCollection col = DCTMBasics.runSelectQuery(dql, session);
dmRecordSet dmRS = new dmRecordSet(col);
col.close();

if (dmRS.isEmpty()) {
    System.out.println("Query returned 0 objects.");
} else {
    System.out.println("Found " + dmRS.getRowCount() + " objects.");
}

// Render the results
if (!dmRS.isEmpty()) {

    // build a single output string with all results
    String output = "<table><tr>";
    
    // get all column names in the record set
    ArrayList<IDfAttr> cols = dmRS.getColumnDefs();

    // print col names as headers
    for (IDfAttr a : cols) {
        output += "<th>" + a.getName() + "</th>";
    }
    output += "</tr>";

    // print record set content under each column heading
    while (dmRS.hasNext()) {
        output += "<tr>";
        IDfTypedObject tObj = dmRS.getNextRow();
        for (IDfAttr a : cols) {
            output += "<td>" + tObj.getString(a.getName()) + "</td>";
        }
        output += "</tr>";
    }
    output += "</table>";
    
    // render the results as an inline HTML table
    display(output, "text/html");
}

_You can run multiple queries by changing the __dql__ variable and rerunning the cell above.  When you are finished, run the cell below to log off of the Documentum server._

In [ ]:
// close session (logout)
if (session != null) {
    DCTMBasics.logoff(session);
    session = null;
}
System.out.println("Logged off");